# Strategy analysis example

Debugging a strategy can be time-consuming. Freqtrade offers helper functions to visualize raw data.
The following assumes you work with SampleStrategy, data for 5m timeframe from Binance and have downloaded them into the data directory in the default location.

## Setup

In [ ]:
import os
from pathlib import Path


# Change directory
# Modify this cell to insure that the output shows the correct path.
# Define all paths relative to the project root shown in the cell output
project_root = "/freqtrade"
i = 0
try:
    os.chdir(project_root)
    if not Path("LICENSE").is_file():
        raise FileNotFoundError("LICENSE file not found")

    while i < 4 and (not Path("LICENSE").is_file()):
        os.chdir(Path(Path.cwd(), "../"))
        i += 1
    project_root = Path.cwd()
    print(Path.cwd())
except FileNotFoundError:
    print("Failed to change directory. Please define the project root in the next cell.")
    project_root = None

In [ ]:
import pprint

import nest_asyncio

from freqtrade.configuration import Configuration
from freqtrade.data.dataprovider import DataProvider
from freqtrade.plugins.pairlistmanager import PairListManager
from freqtrade.resolvers import ExchangeResolver


pp = pprint.PrettyPrinter(indent=2)

nest_asyncio.apply()

# Initialize empty configuration object
config = Configuration.from_files(["user_data/config.json"])

exchange = ExchangeResolver.load_exchange(config=config, validate=True)
pairlist = PairListManager(exchange, config)
dataprovider = DataProvider(config, exchange, pairlist)
print("white-list", pairlist.whitelist)
print("black-list", pairlist.blacklist)

In [ ]:
# Load data using values set above
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType


pair = "BTC/USDT:USDT"
datadir = config["datadir"]

candles = load_pair_history(
    datadir=datadir,
    timeframe=config["timeframe"],
    pair=pair,
    data_format="feather",
    candle_type=CandleType.FUTURES,
)

# Confirm success
print("Loaded " + str(len(candles)) + f" rows of data for {pair} from {datadir}")
candles.head()

## Load and run strategy
* Rerun each time the strategy file is changed

In [ ]:
# Load strategy using values set above
from freqtrade.resolvers import StrategyResolver


strategy = StrategyResolver.load_strategy(config)
strategy.dp = dataprovider

strategy.ft_bot_start()

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles, {"pair": pair})
df.tail()

## Load existing objects into a Jupyter notebook

The following cells assume that you have already generated data using the cli.  
They will allow you to drill deeper into your results, and perform analysis which otherwise would make the output very difficult to digest due to information overload.

### Load backtest results to pandas dataframe

Analyze a trades dataframe (also used below for plotting)

In [ ]:
from freqtrade.data.btanalysis import load_backtest_stats


# You can get the full backtest statistics by using the following command.
# This contains all information used to generate the backtest result.
backtest_dir = "user_data/backtest_results"
strategy = config["strategy"]
stats = load_backtest_stats(backtest_dir)

# All statistics are available per strategy, so if `--strategy-list` was used during backtest, this
# will be reflected here as well.
# Example usages:
pp.pprint(stats["strategy"][strategy]["results_per_pair"])
# Get pairlist used for this backtest
print(stats["strategy"][strategy]["pairlist"])
# Get market change (average change of all pairs from start to end of the backtest period)
print(stats["strategy"][strategy]["market_change"])
# Maximum drawdown ()
print("drawdown_start:", stats["strategy"][strategy]["drawdown_start"])
print("drawdown_end:  ", stats["strategy"][strategy]["drawdown_end"])


# Get strategy comparison (only relevant if multiple strategies were compared)
pp.pprint(stats["strategy_comparison"])

## Analyze

In [ ]:
# Plotting equity line (starting with 0 on day 1 and adding daily profit for each backtested day)

import pandas as pd
import plotly.express as px

from freqtrade.configuration import Configuration
from freqtrade.data.btanalysis import load_backtest_stats


# strategy = 'SampleStrategy'
# config = Configuration.from_files(["user_data/config.json"])
# backtest_dir = config["user_data_dir"] / "backtest_results"

strategy_stats = stats["strategy"][strategy]

df = pd.DataFrame(columns=["dates", "equity"], data=strategy_stats["daily_profit"])
df["equity_daily"] = df["equity"].cumsum()

fig = px.line(df, x="dates", y="equity_daily")
fig.show()

In [202]:
# Load strategy using values set above
from freqtrade.data.dataprovider import DataProvider
from freqtrade.resolvers import StrategyResolver


strategy = StrategyResolver.load_strategy(config)
strategy.dp = DataProvider(config, None, None)
strategy.ft_bot_start()

# Generate buy/sell signals using strategy
# df = strategy.analyze_ticker(candles, {'pair': pair})
# df.tail()